### Suraj Desai Testing Model

In [145]:
import os
import openai
import pandas as pd
openai.api_key = os.getenv("OPENAI_API_KEY")

my_model = "ft:gpt-3.5-turbo-0613:personal:jaarus:8D0z4CHa"

In [101]:
y_pred = []

In [102]:
data = pd.read_excel("dataset/testdata.xlsx")

In [103]:
data.head()

,sentence,query
0,List all the products sold in December. ...,SELECT DISTINCT ProductName FROM Orders_Dece...
1,Get the total revenue generated in December....,SELECT SUM(ValueofSalesUnit * ofSalesUnitSol...
2,Find the total number of orders made by each...,"SELECT CustomerID, COUNT(*) AS TotalOrders F..."
3,Get the names of customers who made pre-orde...,"SELECT FirstName, LastName FROM Customers IN..."
4,List all the menu items with their prices an...,"SELECT Menu_Items.ID, Products_Table.Product..."


In [146]:
def predict(sent):
    completion = openai.ChatCompletion.create(
      model= my_model,
      messages=[
        {"role": "system", "content": "Jaarus is SQL Parser who learns scghema and generates mysql query"},
        {"role": "user", "content": sent[0]}
      ],
      temperature=0.05,
      max_tokens=2048,
      top_p=0.1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return completion.choices[0].message["content"]
    

In [163]:
def get_pred_score(pred):
    
    binary_pred = []

    for test, test_pred in zip(data["query"], pred):
        
        
        if  test.strip().lower().replace(" ", "") == test_pred.strip().lower().replace(" ", ""):
            binary_pred.append(1)
        else:
            binary_pred.append(0)
            
    return binary_pred

#### Testing model trained on self generated dataset 

In [148]:
y_pred = data[["sentence"]].apply(predict, axis=1)

In [164]:
pred = get_pred_score(y_pred)

In [165]:
len(pred)

40

In [166]:
accuracy = pred.count(1) / len(pred)

In [167]:
accuracy

0.525

Accuracy is 52%

#### Testing model trained on self generated + Spider Dataset on 3 epochs

In [170]:
my_model = openai.FineTuningJob.list(limit=2)["data"][0]['fine_tuned_model']

y_pred_spider = data[["sentence"]].apply(predict, axis=1)

In [171]:
pred_spider = get_pred_score(y_pred_spider)

In [173]:
pred_spider.count(1) / len(pred_spider)

0.075

Accuracy : 7.5 % which is terrible

In [183]:
y_pred_spider.tolist()[0], data["query"].tolist()[0]

("  SELECT DISTINCT ProductName FROM Products INNER JOIN Order_Items ON Products.ID = Order_Items.ProductID INNER JOIN Orders ON Order_Items.OrderID = Orders.ID WHERE Orders.OrderDate LIKE '12/%'; ",
 '  SELECT DISTINCT ProductName FROM Orders_December;                        ')

In [99]:
openai.FineTuningJob.list(limit=2)["data"][0]['fine_tuned_model']

'ft:gpt-3.5-turbo-0613:personal:jaarus:8D5w06vK'

In [184]:
y_pred.tolist()[0], data["query"].tolist()[0]

('  SELECT ProductName FROM Orders_December;                                  ',
 '  SELECT DISTINCT ProductName FROM Orders_December;                        ')

In [186]:
data["sentence"][0]

'  List all the products sold in December.                              '

#### Testing model trained on self generated + Spider Dataset on 1 epochs

In [187]:
my_model = openai.FineTuningJob.list(limit=3)["data"][0]['fine_tuned_model']

y_pred_spider_1 = data[["sentence"]].apply(predict, axis=1)

In [188]:
pred_spider_1 = get_pred_score(y_pred_spider)

In [189]:
pred_spider_1.count(1) / len(pred_spider_1)

0.075

Accuracy: 7.5%. Again terrible

In [196]:
openai.FineTuningJob.list(limit=3)["data"][0]['fine_tuned_model']

'ft:gpt-3.5-turbo-0613:personal:jaarus-3:8DFrFQ6R'